In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from utils.data_processing_tick import convert_to_tick
from utils.holiday_utils import remove_friday
from utils.holiday_utils import extract_weekday
notebook_path = os.path.abspath(os.path.join(os.getcwd(), '../notebooks'))
from utils.histdata import import_pair
from utils.data_processing_timestamp import convert_to_every_30min
from utils.data_processing_timestamp import resample_fx_return_by_every_hour
output_file_path = rf'{notebook_path}\data\processed'

**Get Data from histdata.com**

In [2]:
# Get intraday data from histdata.com
%debug
year = 2024
currency = "gbpusd"
import_pair(pair=currency, start=f"{year}01", end=f"{year}12", data_directory=r"data/raw")

ERROR:root:No traceback has been produced, nothing to debug.


Output()

Output()

Output()

Output()

**Create Renko Brick Data**

In [6]:
# Convet_to_tick function : create renko data from intraday historical data
%debug

year = "2024"
ccy = "gbpusd"
brick_size = 0.001 # 0.001 for EURUSD/EURGBP/GBPUSD, 0.1 for USDJPY
use_first_price_as_base=False # True: use first price as base, False: use previous price as base

csv_filepath = rf'{notebook_path}\data\raw\ASCII\M1\{ccy.lower()}\{year}'
csv_filename = f'DAT_ASCII_{ccy.upper()}_M1_{year}.csv'

if (use_first_price_as_base):
    output_filename = f'{year}_{ccy.lower()}_tick_first_price_as_base.csv'
else:
    output_filename = f'{year}_{ccy.lower()}_tick_prev_price_as_base.csv'

convert_to_tick(csv_filepath, csv_filename, output_file_path, output_filename, brick_size, use_first_price_as_base) 

ERROR:root:No traceback has been produced, nothing to debug.


Tick-like data has been saved to tick_like_data.csv
   tick_number            datetime    price  actual_openprice  ticks_moved  \
0            1 2024-01-01 17:46:00  1.27284           1.27287            1   
1            2 2024-01-01 21:38:00  1.27184           1.27171            1   
2            3 2024-01-02 00:43:00  1.27284           1.27290            1   
3            4 2024-01-02 00:53:00  1.27384           1.27395            1   
4            5 2024-01-02 03:07:00  1.27484           1.27506            1   

   directions  fx_return day_of_week  
0           1        NaN      Monday  
1          -1  -0.091133      Monday  
2           1   0.093575     Tuesday  
3           1   0.082489     Tuesday  
4           1   0.087131     Tuesday  


,tick_number,datetime,price,actual_openprice,ticks_moved,directions,fx_return,day_of_week
0,1,2024-01-01 17:46:00,1.27284,1.27287,1,1,NaN,Monday
1,2,2024-01-01 21:38:00,1.27184,1.27171,1,-1,-0.091133,Monday
2,3,2024-01-02 00:43:00,1.27284,1.27290,1,1,0.093575,Tuesday
3,4,2024-01-02 00:53:00,1.27384,1.27395,1,1,0.082489,Tuesday
4,5,2024-01-02 03:07:00,1.27484,1.27506,1,1,0.087131,Tuesday
...,...,...,...,...,...,...,...,...
5135,5136,2024-12-31 12:54:00,1.25184,1.25190,1,1,0.088744,Tuesday
5136,5137,2024-12-31 13:31:00,1.25084,1.25068,1,-1,-0.097452,Tuesday
5137,5138,2024-12-31 14:15:00,1.25184,1.25190,1,1,0.097547,Tuesday
5138,5139,2024-12-31 14:48:00,1.25084,1.25083,1,-1,-0.085470,Tuesday
